# Tutorial: Pump Unit Model with IAPWS Property Package

In this tutorial, we will pump a stream of water using a simple pump unit model, and the IAPWS property package.
The inlet specifications are as follows:

* Flow Rate = 1 kmol/hr
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Molar Enthalpy = 2000 J/mol

The inlet specifications are as follows:
* Pressure Increase = 100000 Pa
* Pump Efficiency = 0.8

## Setting up the problem in IDAES

In [1]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) # dynamic or ss flowsheet needs to be specified here


# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = iapws95.Iapws95ParameterBlock()

In [2]:
# Import pump unit model from the model library
from idaes.generic_models.unit_models.pressure_changer import Pump

# Create an instance of the pump unit, attaching it to the flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.pump = Pump(default={"property_package": m.fs.properties})

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2020-05-14 11:19:34 [ERROR] idaes.generic_models.properties.helmholtz.helmholtz: Library file 'C:\Users\deshpana\AppData\Local\idaes\bin\iapws95_external.so' not found. Was it installed?
2020-05-14 11:19:34 [ERROR] idaes.generic_models.properties.helmholtz.helmholtz: Library file 'C:\Users\deshpana\AppData\Local\idaes\bin\iapws95_external.so' not found. Was it installed?
The initial DOF is 5


In [3]:
assert DOF_initial == 5

In [4]:
# Fix the stream inlet conditions
m.fs.pump.inlet.display()
m.fs.pump.inlet.flow_mol[0].fix(1*1000/3600) # converting to mol/s as unit basis is mol/s
m.fs.pump.inlet.enth_mol[0].fix(iapws95.htpx(T=298.15, P=101325))
m.fs.pump.inlet.pressure[0].fix(101325)
# Fix stream outlet and pump conditions
m.fs.pump.deltaP.fix(100000)
m.fs.pump.efficiency_pump.fix(0.8)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

inlet : Size=1
    Key  : Name     : Value
    None : enth_mol : {0.0: 1000}
         : flow_mol : {0.0: 1}
         : pressure : {0.0: 100000.0}
2020-05-14 11:19:34 [ERROR] idaes.generic_models.properties.helmholtz.helmholtz: Library file 'C:\Users\deshpana\AppData\Local\idaes\bin\iapws95_external.so' not found. Was it installed?
ERROR: evaluating object as numeric value: func_tau_sat(101.325)
        (object: <class
        'pyomo.core.expr.numeric_expr.NPV_ExternalFunctionExpression'>)
    [WinError 126] The specified module could not be found


OSError: [WinError 126] The specified module could not be found

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

In [ ]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.pump.initialize(outlvl=idaeslog.WARNING)
# From the output it can be inferred that since there are no errors or warnings encountered during initialization, nothing is displayed  